In [ ]:
import pandas as pd
import json
import glob
import matplotlib.pyplot as plt
import numpy as np

import sklearn
import sklearn.metrics
import matplotlib
import scipy
import mplhep as hep

import pandas

In [ ]:
def flatten(arr):
    return arr.reshape((arr.shape[0]*arr.shape[1], arr.shape[2]))

In [ ]:
def cms_label(x0=0.12, x1=0.23, x2=0.67, y=0.90):
    plt.figtext(x0, y,'CMS',fontweight='bold', wrap=True, horizontalalignment='left', fontsize=12)
    plt.figtext(x1, y,'Simulation Preliminary', style='italic', wrap=True, horizontalalignment='left', fontsize=10)
    plt.figtext(x2, y,'Run 3 (14 TeV)',  wrap=True, horizontalalignment='left', fontsize=10)

def sample_label(ax, x=0.03, y=0.98):
    plt.text(x, y, "$\mathrm{t}\overline{\mathrm{t}}$ events", va="top", ha="left", size=10, transform=ax.transAxes)


In [ ]:
pid_names = {
    1: "ch.had",
    2: "n.had",
    3: "HFEM",
    4: "HFHAD",
    5: "g",
    6: "el",
    7: "mu"
}

var_names = {
    1: "charge",
    2: "pt",
    3: "eta",
    4: "sin phi",
    5: "cos phi",
    6: "energy"
}

In [ ]:
path = "../experiments/cms-gnn-dense-2cc4e7f9.gpu0.local/"

In [ ]:
Xs = []
ygens = []
ycands = []
ypreds = []
ypreds_raw = []
for fi in glob.glob(path + "/pred*.npz"):
    dd = np.load(fi)
    Xs.append(dd["X"])
    ygens.append(dd["ygen"])
    ycands.append(dd["ycand"])
    ypreds.append(dd["ypred"])
    ypreds_raw.append(dd["ypred_raw"])

X = np.concatenate(Xs)
ygen = np.concatenate(ygens)
ycand = np.concatenate(ycands)
ypred = np.concatenate(ypreds)
ypred_raw = np.concatenate(ypreds_raw)

X_f = X.reshape((X.shape[0]*X.shape[1], X.shape[2]))
ygen_f = ygen.reshape((ygen.shape[0]*ygen.shape[1], ygen.shape[2]))
ycand_f = ycand.reshape((ycand.shape[0]*ycand.shape[1], ycand.shape[2]))
ypred_f = ypred.reshape((ypred.shape[0]*ypred.shape[1], ypred.shape[2]))
ypred_raw_f = ypred_raw.reshape((ypred_raw.shape[0]*ypred_raw.shape[1], ypred_raw.shape[2]))

In [ ]:
dd["X"].shape

In [ ]:
ygen_f.shape, ycand_f.shape, ypred_f.shape, ypred_raw_f.shape, ycand_f.shape

In [ ]:
def load_history(path):
    ret = {}
    for fi in glob.glob(path):
        data = json.load(open(fi))
        epoch = int(fi.split("_")[-1].split(".")[0])
        ret[epoch] = data
    
    max_epoch = max(ret.keys())
    ret2 = []
    for i in range(max_epoch):
        ret2.append(ret[i])
    return pandas.DataFrame(ret2)

In [ ]:
history = load_history(path + "/history_*.json")

In [ ]:
ax = plt.axes()
plt.plot(history["loss"], label="train")
plt.plot(history["val_loss"], label="test")
#plt.yscale("log")
#plt.xlim(50,500)
plt.ylim(history["val_loss"].values[-1]*0.9, history["val_loss"].values[-1]*1.1)
plt.legend(loc="best", frameon=False)
plt.xlabel("epoch")
plt.ylabel("Total loss")
cms_label()
sample_label(ax, x=0.03, y=0.10)
plt.savefig("loss.pdf", bbox_inches="tight")

In [ ]:
ax = plt.axes()
plt.plot(history["cls_loss"], label="train")
plt.plot(history["val_cls_loss"], label="test")
#plt.yscale("log")
#plt.xlim(50,500)
plt.ylim(history["val_cls_loss"].values[-1]*0.9, history["val_cls_loss"].values[-1]*1.1)
plt.legend(loc="best")
plt.xlabel("epoch")
plt.ylabel("Classification loss")
cms_label()
sample_label(ax, x=0.03, y=0.10)
plt.savefig("cls_loss.pdf", bbox_inches="tight")

In [ ]:
ax = plt.axes()
plt.plot(history["energy_loss"], label="train")
plt.plot(history["val_energy_loss"], label="test")
#plt.yscale("log")
#plt.xlim(50,500)
plt.ylim(history["val_energy_loss"].values[-1]*0.9, history["val_energy_loss"].values[-1]*1.1)
plt.legend(loc="best")
plt.xlabel("epoch")
plt.ylabel("Energy loss")
cms_label()
sample_label(ax, x=0.03, y=0.10)
plt.savefig("energy_loss.pdf", bbox_inches="tight")

In [ ]:
ax = plt.axes()
plt.plot(history["pt_loss"], label="train")
plt.plot(history["val_pt_loss"], label="test")
#plt.yscale("log")
#plt.xlim(50,500)
plt.ylim(history["val_pt_loss"].values[-1]*0.9, history["val_pt_loss"].values[-1]*1.1)
plt.legend(loc="best")
plt.xlabel("epoch")
plt.ylabel("Pt loss")
cms_label()
sample_label(ax, x=0.03, y=0.10)
plt.savefig("pt_loss.pdf", bbox_inches="tight")

In [ ]:
plt.plot(history["sin_phi_loss"], label="train")
plt.plot(history["val_sin_phi_loss"], label="test")
#plt.yscale("log")
#plt.xlim(50,500)
plt.ylim(history["val_sin_phi_loss"].values[-1]*0.9, history["val_sin_phi_loss"].values[-1]*1.1)
plt.legend(loc="best")
plt.xlabel("epoch")
plt.ylabel("sin phi loss")
plt.savefig("sin_phi_loss.pdf", bbox_inches="tight")

In [ ]:
ax = plt.axes()
plt.plot(history["cos_phi_loss"], label="train")
plt.plot(history["val_cos_phi_loss"], label="test")
#plt.yscale("log")
#plt.xlim(50,500)
plt.ylim(history["val_cos_phi_loss"].values[-1]*0.9, history["val_cos_phi_loss"].values[-1]*1.1)
plt.legend(loc="best")
plt.xlabel("epoch")
plt.ylabel("cos phi loss")
cms_label()
sample_label(ax, x=0.03, y=0.10)
plt.savefig("cos_phi_loss.pdf", bbox_inches="tight")

In [ ]:
ax = plt.axes()
plt.plot(history["eta_loss"], label="train")
plt.plot(history["val_eta_loss"], label="test")
#plt.yscale("log")
#plt.xlim(50,500)
plt.ylim(history["val_eta_loss"].values[-1]*0.9, history["val_eta_loss"].values[-1]*1.1)
plt.legend(loc="best")
plt.xlabel("epoch")
plt.ylabel("eta loss")
cms_label()
sample_label(ax, x=0.03, y=0.10)
plt.savefig("eta_loss.pdf", bbox_inches="tight")

In [ ]:
ax = plt.axes()
plt.plot(history["charge_loss"], label="train")
plt.plot(history["val_charge_loss"], label="test")
#plt.yscale("log")
#plt.xlim(50,500)
plt.ylim(history["val_charge_loss"].values[-1]*0.9, history["val_charge_loss"].values[-1]*1.1)
plt.legend(loc="best")
plt.xlabel("epoch")
plt.ylabel("charge loss")
cms_label()
sample_label(ax, x=0.03, y=0.10)
plt.savefig("charge_loss.pdf", bbox_inches="tight")

In [ ]:
msk_X = X_f[:, 0]!=0

In [ ]:
x_labels = [
    "track", "PS1", "PS2", "ECAL", "HCAL", "GSF", "BREM", "HFEM", "HFHAD", "SC", "HO"
]
y_labels = [
    "none", "ch.had", "n.had", "HFEM", "HFHAD", "g", "el", "mu"
]

cm0 = sklearn.metrics.confusion_matrix(
    X_f[msk_X, 0],
    ycand_f[msk_X, 0],
    labels=range(12),
#    normalize="true"
)

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.axes()
plt.imshow(cm0[1:12, :8], cmap="Blues", norm=matplotlib.colors.LogNorm())
plt.colorbar()
plt.yticks(ticks=range(11), labels=x_labels);
plt.xticks(ticks=range(8), labels=y_labels);
plt.xlabel("PFCandidate")
plt.ylabel("PFElement")
cms_label(x0=0.2, x1=0.3, x2=0.6, y=0.885)
sample_label(ax, x=0.75, y=1.0)
plt.savefig("pfelem_to_pfcand.pdf", bbox_inches="tight")

In [ ]:
def apply_thresholds_f(thresholds):
    msk = np.ones_like(ypred_raw_f)
    for i in range(len(thresholds)):
        msk[:, i] = ypred_raw_f[:, i]>thresholds[i]
    ypred_id_f = np.argmax(ypred_raw_f*msk, axis=-1)
    return ypred_id_f

def apply_thresholds(thresholds):
    msk = np.ones_like(ypred_raw)
    for i in range(len(thresholds)):
        msk[:, :, i] = ypred_raw[:, :, i]>thresholds[i]
    ypred_id = np.argmax(ypred_raw*msk, axis=-1)
    return ypred_id

In [ ]:
niter = 0
accs = []
def func(thresholds):
    global niter
    #thresholds = np.round(thresholds, 2)
    
    ypred_id = apply_thresholds(thresholds)
    #ypred_id = np.argmax(ypred_raw/thresholds, axis=-1)
    
    err = 0
    for icls in range(0,8):
        ntrue = np.sum(ycand[:, :, 0]==icls, axis=1)
        npred = np.sum(ypred_id==icls, axis=1)
        e = np.sqrt(np.sum((ntrue-npred)**2)) / np.mean(ntrue)
        err += e
    #perm = np.random.permutation(ycand_f[msk_X, 0].shape[0])[:100000]
    #acc = sklearn.metrics.balanced_accuracy_score(ycand_f[msk_X, 0][perm], ypred_id_f[msk_X][perm])
    #acc = sklearn.metrics.jaccard_score(ycand_f[msk_X, 0], ypred_id_f[msk_X], average="macro")
    #err += np.sum((thresholds - 1.0)**2)
    accs.append(err)
    niter += 1
    if niter%10==0:
        print(niter, err, thresholds)
    return err

ret = scipy.optimize.minimize(
    func,
    0.5*np.ones(8),
    method="Nelder-Mead",
    options={"adaptive": True, "xatol": 0.01, "fatol": 0.01}
    #bounds=[(0,2) for i in range(8)]
)

In [ ]:
#thresholds = np.zeros(8)
thresholds = ret.x

In [ ]:
plt.plot(accs) 

plt.ylim(0.8*np.min(accs), 1.2*np.min(accs))

In [ ]:
icls = 1
msk = ycand_f[:, 0] == icls
plt.hist(ypred_raw_f[msk, icls], bins=100, density=1, histtype="step", lw=2);
plt.hist(ypred_raw_f[~msk, icls], bins=100, density=1, histtype="step", lw=2);
plt.yscale("log")

In [ ]:
ypred_id = apply_thresholds(thresholds)
ypred_id_f = apply_thresholds_f(thresholds)

In [ ]:
cm1 = sklearn.metrics.confusion_matrix(
    X_f[msk_X, 0],
    ypred_id_f[msk_X],
    labels=range(11),
)

plt.figure(figsize=(8, 8))
ax = plt.axes()
plt.imshow(cm1[1:12, :8], cmap="Blues", norm=matplotlib.colors.LogNorm())
plt.colorbar()
plt.yticks(ticks=range(11), labels=x_labels);
plt.xticks(ticks=range(8), labels=y_labels);
plt.xlabel("PFCandidate")
plt.ylabel("PFElement")

cms_label(x0=0.2, x1=0.3, x2=0.6, y=0.885)
sample_label(ax, x=0.75, y=1.0)

plt.savefig("pfelem_to_mlpf.pdf", bbox_inches="tight")

In [ ]:
cm_norm = sklearn.metrics.confusion_matrix(
    ycand_f[msk_X, 0],
    ypred_id_f[msk_X],
    labels=range(8),
    normalize="true"
)

cm = sklearn.metrics.confusion_matrix(
    ycand_f[msk_X, 0],
    ypred_id_f[msk_X],
    labels=range(8),
)

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.axes()
plt.imshow(cm_norm, cmap="Blues")
plt.colorbar()

cms_label(x2=0.6, y=0.82)
sample_label(ax, x=0.8, y=1.0)
plt.xticks(range(len(y_labels)), y_labels);
plt.yticks(range(len(y_labels)), y_labels);
plt.xlabel("Predicted PFCandidate")
plt.ylabel("True PFCandidate")

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.axes()
plt.imshow(cm, cmap="Blues", norm=matplotlib.colors.LogNorm())
plt.colorbar()

cms_label(x2=0.6, y=0.82)
sample_label(ax, x=0.8, y=1.0)
plt.xticks(range(len(y_labels)), y_labels);
plt.yticks(range(len(y_labels)), y_labels);
plt.xlabel("Predicted PFCandidate")
plt.ylabel("True PFCandidate")

In [ ]:
bins = {
    2: np.linspace(0,100,100),
    3: np.linspace(-8,8,100),
    4: np.linspace(-1,1,100),
    5: np.linspace(-1,1,100),
    6: np.linspace(0,500,100),
}

In [ ]:
u1 = np.unique(ycand_f[msk_X, 0], return_counts=True)
u2 = np.unique(ypred_id_f[msk_X], return_counts=True)
u3 = np.unique(ygen_f[msk_X, 0], return_counts=True)

In [ ]:
icls = 6
ivar = 2
b = np.linspace(0,100,100)
plt.hist(ygen_f[ygen_f[:, 0]==icls, ivar], bins=b, histtype="step", lw=2);
plt.hist(ycand_f[ycand_f[:, 0]==icls, ivar], bins=b, histtype="step", lw=2);
plt.hist(ypred_f[ypred_id_f==icls, ivar], bins=b, histtype="step", lw=2);
plt.yscale("log")

In [ ]:
plt.bar(u1[0]-0.2, u1[1], width=0.4)
plt.bar(u2[0]+0.2, u2[1], width=0.4)
#plt.bar(u3[0]+0.2, u3[1], width=0.4)
#plt.yscale("log")

In [ ]:
icls = 1
npred = np.sum(ypred_id == icls, axis=1)
ncand = np.sum(ycand[:, :, 0] == icls, axis=1)

In [ ]:
fig, axes = plt.subplots(7, 6, figsize=(6*6,7*5))

for axs, icls in zip(axes, range(1,8)):    
    axes = axs.flatten()
    
    npred = np.sum(ypred_id == icls, axis=1)
    ncand = np.sum(ycand[:, :, 0] == icls, axis=1)
    ngen = np.sum(ygen[:, :, 0] == icls, axis=1)
    
    a = 0.5*min(np.min(npred), np.min(ncand))
    b = 1.5*max(np.max(npred), np.max(ncand))
    
    axes[0].scatter(ncand, npred, marker=".")
    
    axes[0].set_xlim(a,b)
    axes[0].set_ylim(a,b)
    axes[0].plot([a,b],[a,b], color="black", ls="--")
    axes[0].set_title(pid_names[icls])
    axes[0].set_xlabel("number of PFCandidates")
    axes[0].set_ylabel("number of MLPFCandidates")
    
    msk_both = (ycand_f[:, 0]!=0) & (ypred_id_f==icls)
    print(icls, np.sum(msk_both))

    for ivar, ax in zip([2,3,4,5,6], axes[1:]):
        
        hist = np.histogram2d(
            ycand_f[msk_both, ivar],
            ypred_f[msk_both, ivar], bins=(bins[ivar], bins[ivar])
        )
        norm = matplotlib.colors.Normalize(vmin=0, vmax=max(10, np.max(hist[0])))
        if ivar == 2 or ivar == 6:
            norm =  matplotlib.colors.LogNorm(vmin=1, vmax=max(10, 10*np.max(hist[0])))
        hep.hist2dplot(
            hist, cmap="Blues",
            norm=norm,
            ax=ax
        )
        ax.set_title("pred. {}, {}".format(pid_names[icls], var_names[ivar]))
        ax.set_xlabel("true value (PFCandidate)")
        ax.set_ylabel("reconstructed value (MLPF)")
plt.tight_layout()
plt.savefig("full_performance.pdf", bbox_inches="tight")